In [1]:
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
from tqdm import tqdm


from keras.layers import Input
from keras.models import Model, Sequential
from keras.layers.core import Dense, Dropout, Reshape
from keras.layers.advanced_activations import LeakyReLU
from keras.datasets import mnist
from keras.optimizers import Adam
from keras import initializers
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import UpSampling2D
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers.core import Flatten

Using TensorFlow backend.


In [0]:
os.environ["KERAS_BACKEND"] = "tensorflow"

# 실험을 재현하고 동일한 결과를 얻을 수 있는지 확인하기 위해 seed 를 설정합니다.
np.random.seed(10)
# 우리의 랜덤 노이즈 벡터의 차원을 설정합니다.
random_dim = 100


In [0]:
def load_mnist_data():
    # 데이터를 로드합니다.
    (x_train, y_train), (x_test, y_test) = mnist.load_data()
    # 데이터를 -1 ~ 1 사이 값으로 normalize 합니다.
    x_train = (x_train.astype(np.float32) - 127.5)/127.5

    # x_train 의 shape 를 (60000, 28, 28) 에서 (60000, 784) 로 바꿉니다.
    # 따라서 우리는 한 row 당 784 columns 을 가지게 됩니다.
    x_train = x_train.reshape(60000, 28, 28, 1)
    return x_train


In [0]:
def get_optimizer():
    return Adam(lr=0.0002, beta_1=0.5)

In [0]:
def get_generator(optimizer):
  generator = Sequential()
  
  generator.add(Dense(1024, input_dim= random_dim, kernel_initializer=initializers.RandomNormal(stddev=0.02)))
  generator.add(LeakyReLU(0.2))
  
  generator.add(Dense(128*7*7))
  generator.add(BatchNormalization())
  generator.add(LeakyReLU(0.2))

  generator.add(Reshape((7,7,128), input_shape=(128*7*7,)))
  generator.add(UpSampling2D(size=(2,2))) #(14,14,128)
  generator.add(Conv2D(64, (5,5), padding='same')) #(14,14,64)
  generator.add(LeakyReLU(0.2))
  generator.add(UpSampling2D(size=(2,2))) #(28,28,64)
  generator.add(Conv2D(1,(5,5),padding='same')) #(28,28,1)
  generator.add(LeakyReLU(0.2))
  generator.compile(loss='binary_crossentropy', optimizer=optimizer)


  return generator
  


In [0]:
#input = (28,28,1)
def get_discriminator(optimizer):
  discriminator = Sequential()
  discriminator.add(Conv2D(64,(5,5),padding='same',input_shape=(28,28,1))) #(28,28,64)
  discriminator.add(LeakyReLU(0.2))

  discriminator.add(MaxPooling2D(pool_size=(2,2))) #(14,14,64)
  discriminator.add(Conv2D(128,(5,5), padding='same')) #(14,14,128)
  discriminator.add(LeakyReLU(0.2))
  discriminator.add(MaxPooling2D(pool_size=(2,2))) #(7,7,128)
  discriminator.add(Flatten())
  discriminator.add(Dense(1024))
  discriminator.add(LeakyReLU(0.2))
  discriminator.add(Dense(1, activation='sigmoid'))
  discriminator.compile(loss='binary_crossentropy', optimizer=optimizer)

  return discriminator



In [0]:
def get_gan_network(discriminator, generator, optimizer, random_dim=100):
  
  discriminator.trainable = False
  
  gan_input = Input(shape=(random_dim,))
  x = generator(gan_input)
  gan_output = discriminator(x)
  gan = Model(inputs = gan_input, outputs = gan_output)
  gan.compile(loss='binary_crossentropy', optimizer=optimizer)

  return gan


In [0]:
def plot_generated_images(epoch, generator, examples=100, dim=(10, 10), figsize=(10, 10)):
    noise = np.random.normal(0, 1, size=[examples, random_dim])
    generated_images = generator.predict(noise) #(100,28,28,1)
    generated_images = generated_images.reshape(examples, 28, 28)

    plt.figure(figsize=figsize)
    for i in range(generated_images.shape[0]):
        plt.subplot(dim[0], dim[1], i+1)
        plt.imshow(generated_images[i], interpolation='nearest', cmap='gray')
        plt.axis('off')
    plt.tight_layout()
    plt.savefig('gan_generated_image_epoch_%d.png' % epoch)

In [0]:
def train(epochs=1, batch_size=128):
  x_train = load_mnist_data() #(60000,28,28,1)
  batch_count = x_train.shape[0] // batch_size

  adam = get_optimizer()
  generator = get_generator(adam)
  discriminator = get_discriminator(adam)
  gan = get_gan_network(discriminator, generator, adam)

  for e in range(1, epochs + 1):
    print ('-'*15, 'Epoch %d' % e, '-'*15)
    for _ in tqdm(range(batch_count)):
      noise = np.random.normal(0,1,size=(batch_size, random_dim)) #(128, 100)
      
      image_batch = x_train[np.random.randint(0, x_train.shape[0],size=batch_size)] #(128,28,28,1)

      generated_images = generator.predict(noise) #(128, 28, 28, 1)
      X = np.concatenate([image_batch, generated_images]) #(256, 28, 28,1)

      y_dis = np.zeros(2*batch_size)
      y_dis[:batch_size] = 0.9

      discriminator.trainable = True
      discriminator.train_on_batch(X, y_dis)

      noise = np.random.normal(0,1,size=(batch_size, random_dim))
      y_gen = np.ones(batch_size)
      discriminator.trainable = False
      gan.train_on_batch(noise, y_gen)

    if e == 1 or e % 10 == 0:
      plot_generated_images(e, generator)
  
if __name__ == '__main__':
  train(400, 128)





11493376/11490434 [==============================] - 1s 0us/step








Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



  0%|          | 0/468 [00:00<?, ?it/s]

--------------- Epoch 1 ---------------










  0%|          | 2/468 [00:00<00:24, 19.09it/s]

--------------- Epoch 2 ---------------


...

  0%|          | 2/468 [00:00<00:25, 18.21it/s]

--------------- Epoch 131 ---------------


  0%|          | 2/468 [00:00<00:25, 18.52it/s]

--------------- Epoch 132 ---------------


 66%|██████▌   | 310/468 [00:17<00:08, 17.94it/s]

진행 할수록 1 마